## Part 2.2

In [1]:
import csv

In [2]:
int_to_weekday = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}
        
def timestamp_to_weekday_and_hour(timestamp) :
    return int_to_weekday.get(int((timestamp/24) % 7)), timestamp % 24 
print(timestamp_to_weekday_and_hour(30.5))

('Tuesday', 6.5)


This means Tuesday 06:30. This translation will be very practical in attacks.

Let us do an arbitrary search which only keeps queries for clubs performed between midnight and 1am on Tuesdays :

In [3]:
with open('queries.csv', newline='') as csvfile:
    queriesReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    has_header = csv.Sniffer().has_header(csvfile.read(1024))
    if has_header:
        next(queriesReader)
    for row in queriesReader:
        weekday, hour = timestamp_to_weekday_and_hour(float(row[3]))
        if weekday == "Tuesday" and hour < 1 and row[4] == "club" :
            print(', '.join(row))

210.68.124.185, 46.56336871719933, 6.648431242252003, 24.000999171226507, club
197.134.251.204, 46.51384877401542, 6.638021175914131, 192.158234557433, club
250.71.124.231, 46.55215382261362, 6.604171825604082, 192.47575231364215, club
247.196.95.171, 46.544409611768295, 6.579111041539487, 192.55236588701695, club


As expected, there are not many results. Those 4 people could prove to be ideal victims of a ransomware attack. Let's try to learn more about the first user by filtering on its IP as well as only keeping queries of cafeterias :

In [4]:
with open('queries.csv', newline='') as csvfile:
    queriesReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in queriesReader:
        if row[0] == "210.68.124.185" and row[4] == "cafeteria":
            print(', '.join(row))
            print(timestamp_to_weekday_and_hour(float(row[3])))

210.68.124.185, 46.53976081739804, 6.574063213164034, 11.702267726620121, cafeteria
('Monday', 11.702267726620121)
210.68.124.185, 46.53976081739804, 6.574063213164034, 34.50672801621601, cafeteria
('Tuesday', 10.506728016216009)
210.68.124.185, 46.53976081739804, 6.574063213164034, 59.71451503914215, cafeteria
('Wednesday', 11.71451503914215)
210.68.124.185, 46.53976081739804, 6.574063213164034, 84.38001956058253, cafeteria
('Thursday', 12.380019560582525)
210.68.124.185, 46.53976081739804, 6.574063213164034, 108.59528266831816, cafeteria
('Friday', 12.595282668318163)
210.68.124.185, 46.53976081739804, 6.574063213164034, 180.02441857824775, cafeteria
('Monday', 12.024418578247747)
210.68.124.185, 46.53976081739804, 6.574063213164034, 203.04560260796723, cafeteria
('Tuesday', 11.045602607967226)
210.68.124.185, 46.53976081739804, 6.574063213164034, 227.94540536756293, cafeteria
('Wednesday', 11.945405367562927)
210.68.124.185, 46.53976081739804, 6.574063213164034, 252.43632281906264, 

Since these searches cover each of the 5 working days and are always sourced at the exact same location around lunch time, we can confidently say we know exactly where this user work. In this example, assume we now know its employer. Let us now try to find out where the user lives by filtering on its "supermarket" queries :

In [5]:
with open('queries.csv', newline='') as csvfile:
    queriesReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in queriesReader:
        if row[0] == "210.68.124.185" and row[4] == "supermarket":
            print(', '.join(row))
            print(timestamp_to_weekday_and_hour(float(row[3])))

210.68.124.185, 46.56920952755545, 6.64953112929113, 17.660631454803113, supermarket
('Monday', 17.660631454803113)
210.68.124.185, 46.56920952755545, 6.64953112929113, 43.62313486438009, supermarket
('Tuesday', 19.62313486438009)
210.68.124.185, 46.56336871719933, 6.648431242252003, 24.000999171226507, supermarket
('Tuesday', 0.0009991712265069452)
210.68.124.185, 46.56596058841714, 6.642228369125065, 69.31305252574826, supermarket
('Wednesday', 21.31305252574826)
210.68.124.185, 46.56920952755545, 6.64953112929113, 89.10384857262238, supermarket
('Thursday', 17.103848572622383)
210.68.124.185, 46.56596058841714, 6.642228369125065, 94.4445729732377, supermarket
('Thursday', 22.444572973237698)
210.68.124.185, 46.56920952755545, 6.64953112929113, 113.61852528201781, supermarket
('Friday', 17.618525282017814)
210.68.124.185, 46.56596058841714, 6.642228369125065, 128.04058654198784, supermarket
('Saturday', 8.040586541987835)
210.68.124.185, 46.56596058841714, 6.642228369125065, 188.6318

We learn of 2 new locations which are close to each other (700m according to Google Maps), which is weird and is probably due to the fact that this is a simulated dataset. We see that weekend days are included : we are confident this is where the user lives. Identifying him thanks to his address and its workplace should be easy, though in this case slightly complicated because of the fact that there are 2 different locations. We could, for the sake of the example, assume that the rarer location is the house of the user's best friend for example, and would necessitate checking both adresses until we successfully find an employee of the previously identified company/employer. We could then threaten him to reveal his clubbing habits to its employer for example. Let us see this habit in more detail :

In [6]:
with open('queries.csv', newline='') as csvfile:
    queriesReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in queriesReader:
        if row[0] == "210.68.124.185" and row[4] == "club":
            print(', '.join(row))
            print(timestamp_to_weekday_and_hour(float(row[3])))

210.68.124.185, 46.56920952755545, 6.64953112929113, 17.660631454803113, club
('Monday', 17.660631454803113)
210.68.124.185, 46.56336871719933, 6.648431242252003, 24.000999171226507, club
('Tuesday', 0.0009991712265069452)
210.68.124.185, 46.56596058841714, 6.642228369125065, 69.31305252574826, club
('Wednesday', 21.31305252574826)
210.68.124.185, 46.56596058841714, 6.642228369125065, 117.90671732023733, club
('Friday', 21.906717320237334)
210.68.124.185, 46.56920952755545, 6.64953112929113, 153.6845968833121, club
('Sunday', 9.68459688331211)
210.68.124.185, 46.56920952755545, 6.64953112929113, 209.74748534599888, club
('Tuesday', 17.747485345998882)
210.68.124.185, 46.56920952755545, 6.64953112929113, 235.04178799572566, club
('Wednesday', 19.041787995725656)
210.68.124.185, 46.56336871719933, 6.648431242252003, 297.37760167993076, club
('Saturday', 9.377601679930763)
210.68.124.185, 46.56336871719933, 6.648431242252003, 320.9086966175637, club
('Sunday', 8.908696617563692)
210.68.12

This user seems to go clubbing almost every day which is information that should be kept private. We can see that it is very easy and fast to mount such attacks, especially considering this is only focused on one user and proved quite successful.

Inferring relationships between users is harder however. In a realistic setting, only one person of a group should be using the app to look for a destination and this seems to be reflected in the data : we did not find multiple users using the app at the exact same time and place. The closest we got to such a find was the following :

In [7]:
with open('queries.csv', newline='') as csvfile:
    queriesReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in queriesReader:
        if row[1].startswith("46.54") and row[2].startswith("6.57") and row[3].startswith("373.25"):
            print(', '.join(row))

42.58.33.7, 46.54648429922497, 6.57344706420223, 373.25647094335324, cafeteria
42.58.33.7, 46.54648429922497, 6.57344706420223, 373.25647094335324, restaurant
139.251.47.207, 46.54637740125457, 6.575353054770723, 373.2569389035634, cafeteria
139.251.47.207, 46.54637740125457, 6.575353054770723, 373.2569389035634, restaurant
65.106.64.1, 46.54242230623346, 6.577281728790473, 373.2541769608664, cafeteria
65.106.64.1, 46.54242230623346, 6.577281728790473, 373.2541769608664, restaurant


In [8]:
print(timestamp_to_weekday_and_hour(373.25))

('Tuesday', 13.25)


Here, the first and second users are 230m apart and do the same search at almost the same time. However, this is likely a coincidence especially considering how normal it is to look for cafeterias and restaurants at 13:15, even if they were in a fast moving vehicle when doing the searches which would explain why the searches are 230m apart in the hypothetical case where they were friends looking for a meal together. The third user is even further away location wise. 

In conclusion, we would not spend our time and resources trying to mount attacks trying to infer relationships if we were adversaries and would focus on "simpler" attacks which target one user at a time.

## Part 2.3

The defense described in the report should ideally be very smart by producing very realistic random queries so that those random queries are indistinguishable from real ones even by a smart adversary. The interaction between the various fields and between multiple queries would need to be taken into account. For example, multiple fake queries should reuse the exact same location so that an attacker would be led to believe that it is the user's home or worplace. If they were truly random this would not happen and the attacker could easily single out the only queries with repeating locations and conclude that they are real. This logic also applies to other sensitive fields such as the timestamp : the user could wish to hide at which time of the day they are using the app. The defense would then need to produce queries at various times of the day while still being realistic : a smart attacker would discard queries for dojos in a rural area at midnight for example.

Since the generation of truly indistinguishable randomized queries from a "perfectly smart" attacker is a very complex problem which goes beyond the scope of this defense evaluation, our experiment will consist in a more naive implementation which only uses the strategy to hide the location data of the user. We want to achieve a privacy level of 0.75 (arbitrarily chosen for this experiment) according to the definition described in the report, assuming the only data we consider sensitive is the location against a strategic adversary. 0.75 is very low but will make things far more readable while illustrating the idea. This is thus a subset of the defense described in the report but its analysis produces results which are also applicable to the hypothetical "perfectly smart" solution.

For this experiment, we also assume that every client is using the defense which is more interesting. In the case where only some of them do, an attacker could easily realise who is defending themself from the volume of queries and focus their efforts on the users who do not use the defense.

To simplify the implementation, the generation of fake queries will be procedural and mirror the existing queries except for their location. To achieve a privacy of 0.75, we will thus create 3 fake queries from each real one, only changing the location deterministically which guarantees realism. This deterministic change is intentionally simple : we only change the longitude so that the 4 queries (1 is real and 3 are generated from it) have longitudes as different as possible while still being in the grid. The number of queries is thus multiplied by 4, which increases the workload on the user's device and on the provider's server, which is our utility drawback.

Let us take a look at how the previous attacks would look like with such a defense in place :

In [18]:
def print_real_and_3_fakes(row):
    print(', '.join(row))
    for _ in range(3):
        row2 = float(row[2])
        row2 += 0.025
        if row2 > 6.65:
            row2 -= 0.1
        row[2] = str(row2)
        print(', '.join(row))

with open('queries.csv', newline='') as csvfile:
    queriesReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    has_header = csv.Sniffer().has_header(csvfile.read(1024))
    if has_header:
        next(queriesReader)
    for row in queriesReader:
        weekday, hour = timestamp_to_weekday_and_hour(float(row[3]))
        if weekday == "Tuesday" and hour < 1 and row[4] == "club" :
            print_real_and_3_fakes(row)

210.68.124.185, 46.56336871719933, 6.648431242252003, 24.000999171226507, club
210.68.124.185, 46.56336871719933, 6.573431242252004, 24.000999171226507, club
210.68.124.185, 46.56336871719933, 6.598431242252004, 24.000999171226507, club
210.68.124.185, 46.56336871719933, 6.623431242252004, 24.000999171226507, club
197.134.251.204, 46.51384877401542, 6.638021175914131, 192.158234557433, club
197.134.251.204, 46.51384877401542, 6.563021175914132, 192.158234557433, club
197.134.251.204, 46.51384877401542, 6.5880211759141325, 192.158234557433, club
197.134.251.204, 46.51384877401542, 6.613021175914133, 192.158234557433, club
250.71.124.231, 46.55215382261362, 6.604171825604082, 192.47575231364215, club
250.71.124.231, 46.55215382261362, 6.629171825604082, 192.47575231364215, club
250.71.124.231, 46.55215382261362, 6.554171825604083, 192.47575231364215, club
250.71.124.231, 46.55215382261362, 6.579171825604083, 192.47575231364215, club
247.196.95.171, 46.544409611768295, 6.579111041539487, 

Instead of 4 queries, an attacker would now obtain 16. We of course assume that the attacker would not necessarily retrieve them in this specific order where the first query of a group of 4 is always the real one. Notice how this defense does not even try to hide that some queries are fake, which is fine since we assume the adversary to be aware of the defense mechanism anyway. Also recall that for this simplified defense experiment, we only consider the location to be a sensitive information : an ideal "perfectly smart solution" achieving a privacy of 0.75 would produce queries with every field changing except the IP while still being realistic by keeping all interactions into account. However, in the case where the client truly only cares about hiding his location and nothing else, this kind of simple implementation could be sufficient in the case where the grid only covers realistic locations (no lakes, etc). An attacker thus suspects that every query could be real, but since they are indistinguishable he only has a 25% chance of being right for a particular one. 

Let us now look at the next step of the previous, multi-step attack on the first IP by filtering queries on cafeterias :

In [21]:
def print_real_and_3_fakes_with_time(row):
    print(', '.join(row))
    print(timestamp_to_weekday_and_hour(float(row[3])))
    for _ in range(3):
        row2 = float(row[2])
        row2 += 0.025   # quarter of the longitude length of the grid
        if row2 > 6.65:   #rightmost longitude of the grid
            row2 -= 0.1   # the longitude length of the grid
        row[2] = str(row2)
        print(', '.join(row))
        print(timestamp_to_weekday_and_hour(float(row[3])))

with open('queries.csv', newline='') as csvfile:
    queriesReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in queriesReader:
        if row[0] == "210.68.124.185" and row[4] == "cafeteria":
            print_real_and_3_fakes_with_time(row)

210.68.124.185, 46.53976081739804, 6.574063213164034, 11.702267726620121, cafeteria
('Monday', 11.702267726620121)
210.68.124.185, 46.53976081739804, 6.599063213164034, 11.702267726620121, cafeteria
('Monday', 11.702267726620121)
210.68.124.185, 46.53976081739804, 6.6240632131640345, 11.702267726620121, cafeteria
('Monday', 11.702267726620121)
210.68.124.185, 46.53976081739804, 6.649063213164035, 11.702267726620121, cafeteria
('Monday', 11.702267726620121)
210.68.124.185, 46.53976081739804, 6.574063213164034, 34.50672801621601, cafeteria
('Tuesday', 10.506728016216009)
210.68.124.185, 46.53976081739804, 6.599063213164034, 34.50672801621601, cafeteria
('Tuesday', 10.506728016216009)
210.68.124.185, 46.53976081739804, 6.6240632131640345, 34.50672801621601, cafeteria
('Tuesday', 10.506728016216009)
210.68.124.185, 46.53976081739804, 6.649063213164035, 34.50672801621601, cafeteria
('Tuesday', 10.506728016216009)
210.68.124.185, 46.53976081739804, 6.574063213164034, 59.71451503914215, cafet

The attacker's task of identifying where the user works is now 4 times as hard since there are 4 locations which are strong contenders.

Let us now look at the supermarket filter to find out where the user lives :

In [22]:
with open('queries.csv', newline='') as csvfile:
    queriesReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in queriesReader:
        if row[0] == "210.68.124.185" and row[4] == "supermarket":
            print_real_and_3_fakes_with_time(row)

210.68.124.185, 46.56920952755545, 6.64953112929113, 17.660631454803113, supermarket
('Monday', 17.660631454803113)
210.68.124.185, 46.56920952755545, 6.574531129291131, 17.660631454803113, supermarket
('Monday', 17.660631454803113)
210.68.124.185, 46.56920952755545, 6.599531129291131, 17.660631454803113, supermarket
('Monday', 17.660631454803113)
210.68.124.185, 46.56920952755545, 6.624531129291132, 17.660631454803113, supermarket
('Monday', 17.660631454803113)
210.68.124.185, 46.56920952755545, 6.64953112929113, 43.62313486438009, supermarket
('Tuesday', 19.62313486438009)
210.68.124.185, 46.56920952755545, 6.574531129291131, 43.62313486438009, supermarket
('Tuesday', 19.62313486438009)
210.68.124.185, 46.56920952755545, 6.599531129291131, 43.62313486438009, supermarket
('Tuesday', 19.62313486438009)
210.68.124.185, 46.56920952755545, 6.624531129291132, 43.62313486438009, supermarket
('Tuesday', 19.62313486438009)
210.68.124.185, 46.56336871719933, 6.648431242252003, 24.0009991712265

Instead of having only 2 locations which are close to each other, there are now 4 such pairs.

The solution achieved the low goal of 0.75 privacy for the location data, which only quadruples the amount of queries. Because of the light weight of queries, much higher privacies could be feasible. This kind of defense is especially strong when the adversary has no way to check each location with a reliable way to confirm if it is correct (which would require additional information) : such an attacker must bring themself to accept that any of the locations could be the correct one with no certainty.

However, if the attacker has a way to verify each query, then this defense only multiplies the amount of work they must do to figure out the real information. This work scales linearly with the amount of fake queries produced (as long as they are realistic which is the biggest challenge). However, in a realistic context, generating one fake query should be  cheaper for the client than the verification process of each query on the part of the attacker, which requires "real world" knowledge. We thus tend to think that this is very much a viable strategy in general. The biggest challenge resides in randomly generating realistic queries, which would require much more complex experiments.